In [ ]:
!pip install openai langchain tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 756.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.3 MB/s 

In [ ]:
# importing all the dependencies
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.cache import InMemoryCache
import langchain
import chromadb
import tiktoken
import pandas as pd
import os

In [ ]:
f = open("/content/api_key.txt")
api_key = f.read()
chat = ChatOpenAI(openai_api_key=api_key, temperature=0)

In [ ]:
df = pd.read_csv("/content/sdlms_dev.performance_reviews (2).csv")
df.head()

In [ ]:
df["name"]

0    TARUGU UMA MAHESWAR REDDY
1                  Anaya Sutar
2               Tushar Agarwal
3               Hinglaj Tanwar
4                  Yeleza_Tryp
5                 Fardin Kamal
6                Samidha Kumar
7                prajesheleven
8                 Harshvardhan
9                 Gaurav Dahal
Name: name, dtype: object

In [ ]:
output_format = """

WEEKLY REPORT:
Summary:

Bottlenecks:

Suggestions
Personalized Plan to Address and Fix Key Employee Weaknesses:

"""

In [ ]:
def weekly_report(x):

  candidiate_fact_sheet = df.iloc[x]
  answers = candidiate_fact_sheet[3:-6]
  name = candidiate_fact_sheet[1]

  output_format = """

  WEEKLY REPORT:
  Summary:

  Bottlenecks:

  Suggestions
  Personalized Plan to Address and Fix Key Employee Weaknesses:

  """

  question_list = [
  "How would you quantify your contributions (of ...",
  "What are the expected figures?",
  "What are the actual figures?",
  "Did you understand how your project lead quant...",
  "What bottlenecks are slowing you down?",
  "Why do you think you're facing this bottleneck?",
  "Would you like to create a process to solve th...",
  "What are your milestones for the next week?",
  "Why have you set these milestones?",
  "How are you improving the quality and quantity...",
  "How do you plan to achieve these milestones?",
  "How are you assessing your work?",
  "Do you rely on senior feedback or do you have ...",
  "If your project has a long gestation period an...",
  ]

  template_string = """Based on the {name}, the intern's performance review \
  raise only 3-4 serious red-flags in bullet-points \
  using the question's list to the answers.\nQuestions:{question_list}\nAnswers:{answers}
  Do not mentions questions in the summary.
  Raise atmost 4 red-flags.
  """
  prompt_template = ChatPromptTemplate.from_template(template_string)
  summary = prompt_template.format_messages(name = name,
                                            answers = answers,
                                            question_list = question_list)
  response = chat(summary)
  template1 = "Give a summary of this intern's performance review\n{review}"
  prompt1 = ChatPromptTemplate.from_template(template1)
  chain1 = LLMChain(llm=chat, prompt = prompt1, output_key="review_summary")
  template2 = "Identify Key employee weakness in this review summary\n{review_summary}"
  prompt2 = ChatPromptTemplate.from_template(template2)
  chain2 = LLMChain(llm=chat, prompt = prompt2, output_key="weaknesses")
  template3 = "Create a personalized plan 'Personalized Plan to Address and Fix Key Employee Weaknesses:' to help address and fix these weaknesses with at most 4 pointers.\n{weaknesses}"
  prompt3 = ChatPromptTemplate.from_template(template3)
  chain3 = LLMChain(llm=chat, prompt = prompt3, output_key="final_plan")

  # Setting up SequentialChains
  seq_chain = SequentialChain(chains=[chain1, chain2, chain3],
                              input_variables=["review"],
                              output_variables=["review_summary","weaknesses","final_plan"],
                              verbose=False)
  results = seq_chain(candidiate_fact_sheet)
  print("WEEKLY REPORT:")
  print("\n")
  print("Summary:")
  print(results['review_summary'])
  print("\n")
  print("Bottlenecks:")
  print(response.content)
  print("\n")
  print("Suggestions")
  print(results["final_plan"])

In [ ]:
weekly_report(4)

WEEKLY REPORT:


Summary:
This intern's performance review indicates that they have completed more reviews than expected, with a total of 7 out of 4 reviews completed. They have also finished the UBS and Tri-data tasks as planned. The intern acknowledges that the unavailability of some team members has caused bottlenecks in their work. To address this issue, they plan to delegate work to the team. The milestones for this intern include completing 4 reviews and creating reports for client companies. The intern understands the importance of reviews for team collaboration and is working on collecting data for the Tri-data task and implementing a report system. They are also trying to optimize and improve the overall process. The intern's ability to complete tasks on time and their improvement efforts are being assessed, and they are still awaiting feedback from seniors. The intern documents feedback and reasoning for long-gestation tasks. Overall, the intern's performance is satisfactory,

WEEKLY REPORT:


Summary:
This intern's performance review indicates that they have completed more reviews than expected, with a total of 7 out of 4 reviews completed. They have also finished the UBS and Tri-data tasks. The intern acknowledges that there were bottlenecks in the previous week due to the unavailability of some team members. To address this issue, they plan to delegate work to the team. The milestones for the intern include completing 4 reviews and creating reports for client companies. The intern understands the importance of reviews with the teams and is working on collecting data for the Tri-data task and implementing a report system. They are also trying to optimize and improve the overall process. The intern's ability to complete tasks on time and their improvement efforts are being assessed, and they are still awaiting feedback from seniors. The intern documents feedback and reasoning for long-gestation tasks. Overall, the intern's performance is satisfactory, with room for improvement and feedback from seniors.


Bottlenecks:
1. The intern's actual figures for their contributions are lower than the expected figures.
2. The intern is facing a bottleneck due to the unavailability of some team members.
3. The intern plans to improve the quality and quantity of their work by optimizing and improving processes.
4. The intern relies on senior feedback for assessing their work.


Suggestions
Personalized Plan to Address and Fix Key Employee Weaknesses:

1. Provide training and guidance on effective delegation techniques: Schedule a one-on-one meeting with the intern to discuss the importance of delegation and provide them with resources and training materials on how to delegate tasks effectively. Offer guidance on identifying suitable tasks for delegation, setting clear expectations, and providing necessary support to team members.

2. Assign a mentor or coach: Pair the intern with a more experienced team member who can serve as a mentor or coach. This individual can provide ongoing support and guidance, helping the intern develop their delegation skills through practical examples and feedback.

3. Encourage regular check-ins and feedback sessions: Establish a system for regular check-ins and feedback sessions to monitor the intern's progress in delegating tasks. During these sessions, provide constructive feedback on their delegation efforts, highlighting areas of improvement and offering suggestions for refinement.

4. Provide opportunities for practice: Assign the intern with small projects or tasks that require delegation. This will allow them to practice their newly acquired skills in a controlled environment. Monitor their progress and provide feedback to help them refine their delegation techniques.

By implementing this personalized plan, the intern will have the opportunity to develop and improve their delegation skills, enabling them to effectively manage and distribute workload in the future.

WEEKLY REPORT:


Summary:
This intern, Fardin Kamal, has performed well in their role as a front-end developer. They have created three new features and made two modifications to the front-end. In addition, they have fixed the UI of Performance Scoop and Performance Review and created a new history feature. They have also been able to negotiate with the project lead about priorities and have made progress towards finishing the PDGMS dashboard's backend. The intern has received positive feedback and is planning to continue improving based on feedback. They are seen as reliable by senior members of the team and have shown the ability to work on long-gestation projects. Overall, their performance has been satisfactory.


Bottlenecks:
1. Limited developers causing bottlenecks in the project.
2. The intern is facing a bottleneck due to a lack of hiring more developers.
3. The intern would like to create a process to solve the bottleneck issue.
4. The intern's milestones for the next week include finishing the PDGMS dashboard's backend.


Suggestions
Personalized Plan to Address and Fix Key Employee Weaknesses:

1. Conduct a performance review: Schedule a one-on-one meeting with Fardin Kamal to discuss their performance and identify any areas where they may need improvement. This will provide an opportunity for open communication and allow them to express any concerns or challenges they may be facing.

2. Set clear expectations and goals: Clearly define the expectations and goals for Fardin Kamal's role as a front-end developer. This will help them understand what is expected of them and provide a clear direction for improvement. Regularly review and update these goals to ensure they are aligned with the company's objectives.

3. Provide training and development opportunities: Identify any specific skills or knowledge gaps that Fardin Kamal may have and provide them with the necessary training and development opportunities to address these weaknesses. This could include attending workshops, online courses, or mentoring programs to enhance their skills and knowledge in areas where improvement is needed.

4. Offer constructive feedback and support: Continuously provide constructive feedback to Fardin Kamal to help them identify areas for improvement and provide guidance on how to address these weaknesses. Offer support and resources to help them overcome any challenges they may be facing and encourage them to seek feedback from their peers and colleagues.

By implementing this personalized plan, you can help Fardin Kamal address and fix any weaknesses they may have, ultimately improving their performance and contributing to the overall success of the team and organization.

In [ ]:
question_list = [
"How would you quantify your contributions (of ...",
"What are the expected figures?",
"What are the actual figures?",
"Did you understand how your project lead quant...",
"What bottlenecks are slowing you down?",
"Why do you think you're facing this bottleneck?",
"Would you like to create a process to solve th...",
"What are your milestones for the next week?",
"Why have you set these milestones?",
"How are you improving the quality and quantity...",
"How do you plan to achieve these milestones?",
"How are you assessing your work?",
"Do you rely on senior feedback or do you have ...",
"If your project has a long gestation period an...",
]

In [ ]:
question_list[1:3]

['What are the expected figures?', 'What are the actual figures?']

In [ ]:
candidiate_fact_sheet = df.iloc[2]
answers = candidiate_fact_sheet[4:6]
answers

expected    1. Make sure the team gets done with the user ...
actual      1. The team got done with the user flow and ma...
Name: 2, dtype: object

In [ ]:
def weekly_report(x):

  candidiate_fact_sheet = df.iloc[x]
  answers = candidiate_fact_sheet[3:-6]
  deliverables = candidiate_fact_sheet[4:6]
  name = candidiate_fact_sheet[1]

  question_list = [
  "How would you quantify your contributions (of ...",
  "What are the expected figures?",
  "What are the actual figures?",
  "Did you understand how your project lead quant...",
  "What bottlenecks are slowing you down?",
  "Why do you think you're facing this bottleneck?",
  "Would you like to create a process to solve th...",
  "What are your milestones for the next week?",
  "Why have you set these milestones?",
  "How are you improving the quality and quantity...",
  "How do you plan to achieve these milestones?",
  "How are you assessing your work?",
  "Do you rely on senior feedback or do you have ...",
  "If your project has a long gestation period an...",
  ]

  questions_d = question_list[1:3]

  # for bottlenecks
  template_string = """Based on the {name}, the intern's performance review \
  raise only 3-4 serious red-flags in bullet-points \
  using the question's list to the answers.\nQuestions:{question_list}\nAnswers:{answers} \
  Do not mentions questions in the summary.
  Raise atmost 4 red-flags.
  """
  prompt_template = ChatPromptTemplate.from_template(template_string)
  summary = prompt_template.format_messages(name = name,
                                            answers = answers,
                                            question_list = question_list
                                            )
  response = chat(summary)

  # for deliverables
  template_string_d = """Based on the {name}, the intern's performance review \
  raise only 3-4  bullet-points \
  using the question's list to the answers.\nQuestions:{question_list}\nAnswers:{answers} \
  Do not mentions questions.
  """

  prompt_template_d = ChatPromptTemplate.from_template(template_string_d)
  summary_d = prompt_template.format_messages(name = name,
                                            answers = deliverables,
                                            question_list = questions_d
                                            )
  response_d = chat(summary_d)

  # SequentialChain
  template1 = "Give a summary of this intern's performance review\n{review}"
  prompt1 = ChatPromptTemplate.from_template(template1)
  chain1 = LLMChain(llm=chat, prompt = prompt1, output_key="review_summary")
  template2 = "Identify Key employee weakness in this review summary\n{review_summary}"
  prompt2 = ChatPromptTemplate.from_template(template2)
  chain2 = LLMChain(llm=chat, prompt = prompt2, output_key="weaknesses")
  template3 = "Create a personalized plan 'Personalized Plan to Address and Fix Key Employee Weaknesses:' to help address and fix these weaknesses with at most 4 pointers.\n{weaknesses}"
  prompt3 = ChatPromptTemplate.from_template(template3)
  chain3 = LLMChain(llm=chat, prompt = prompt3, output_key="final_plan")

  # Setting up SequentialChains
  seq_chain = SequentialChain(chains=[chain1, chain2, chain3],
                              input_variables=["review"],
                              output_variables=["review_summary", "weaknesses", "final_plan"],
                              verbose=False)
  results = seq_chain(candidiate_fact_sheet)
  print("WEEKLY REPORT:")
  print("\n")
  print("Summary:")
  print(results['review_summary'])
  print("\n")
  print("Deliverables:")
  print(response_d.content)
  print("\n")
  print("Bottlenecks:")
  print(response.content)
  print("\n")
  print("Suggestions")
  print(results["final_plan"])

In [ ]:
weekly_report(4)

WEEKLY REPORT:


Summary:
This intern's performance review indicates that they have completed more reviews than expected, with a total of 7 out of 4 reviews completed. They have also finished working on the UBS and Tri-data projects. The intern acknowledges that there were bottlenecks in the previous week due to the unavailability of some team members. To address this issue, they plan to delegate work to the team. The milestones for the intern include completing 4 reviews and creating reports for client companies. The intern understands the importance of reviews with the teams and is working on collecting data for the Tri-data project and improving the report system. The intern is assessed based on their ability to complete tasks on time and their improvement efforts. They still rely on feedback from seniors and document feedback and reasoning. Overall, the intern's performance is satisfactory, and they are actively working on improving their processes.


Deliverables:
- The intern did

WEEKLY REPORT:


Summary:
This intern, Hinglaj Tanwar, has been performing well in their role. They have been actively involved in creating and reviewing content, as well as conducting interviews. They have met the expected targets for these tasks. However, they have identified a bottleneck in their performance, which is the inability to reach out to enough people due to a stack of priority tasks. They have suggested creating a schedule to address this issue. The milestones set for the intern include reaching out to 1000 individuals and sending 500 emails. These milestones were discussed with the team and are considered key performance indicators. The intern's performance will be assessed based on feedback and whether they are able to complete tasks on time. Both the intern and senior staff rely on each other for support. The intern acknowledges that some tasks may have a long gestation period. This performance review covers the week of August 7th to August 13th, 2023.


Deliverables:
- The expected figures for creating and reviewing content were 10, but the actual figures are not mentioned.
- The expected figures for taking initiative were not mentioned, and the actual figures are also not mentioned.
- The expected figures for problem-solving were not mentioned, and the actual figures are also not mentioned.
- The expected figures for teamwork were not mentioned, and the actual figures are also not mentioned.


Bottlenecks:
- The intern's actual figures for their contributions are the same as the expected figures.
- The intern is facing a bottleneck in not being able to outreach enough people due to a stack of priority tasks.
- The intern has created a process by creating a schedule to solve this bottleneck.
- The intern's milestones for the next week include reaching out to 1000 people, sending 500 emails, and hiring 2 interns.


Suggestions
Personalized Plan to Address and Fix Key Employee Weaknesses:

1. Prioritize and Delegate Tasks: Help the intern identify and prioritize their tasks based on urgency and importance. Encourage them to delegate non-essential tasks to other team members or seek assistance when necessary. This will free up their time and allow them to focus on reaching out to more people.

2. Time Management and Planning: Work with the intern to develop effective time management strategies. Help them create a daily or weekly schedule that includes dedicated time for reaching out to people. Encourage them to set specific goals and deadlines for each outreach task to ensure they stay on track.

3. Improve Communication Skills: Provide the intern with resources and training opportunities to enhance their communication skills. This can include workshops, online courses, or mentoring sessions. Focus on improving their ability to effectively communicate their needs, prioritize tasks, and seek assistance when required.

4. Regular Check-ins and Support: Schedule regular check-ins with the intern to monitor their progress and provide guidance. Offer support and guidance whenever they face challenges or feel overwhelmed. By maintaining open lines of communication, you can address any issues promptly and provide the necessary support to help them overcome their weaknesses.

By implementing this personalized plan, the intern will be able to address their weakness of not reaching out to enough people due to a stack of priority tasks. This will ultimately improve their performance and contribute to the overall success of the team.